# Team GenZ 

In [63]:
from googleapiclient.discovery import build
import pandas as pd
import seaborn as sns

In [65]:
api_key = 'Enter your api key here'

channel_ids = [ # Some Educational Channels
               'UCF7BExjT2zH_mmyqOB139Dg', # Apni Kaksha
               'UCVtL0esgwmD3bEZ2iNUjzHw', # Apni Kaksha NEET
               'UCmXZxX_qexEZxhb5_vQKPCw', # Aman Dhattarwal
               'UCBwmMxybNva6P_5VmxjzwqA', # Apna College
               'UCUNe-AifjeUt9QRn7lkgJ1g', # Apni Kaksha - 9th & 10th
            
               'UC7btqG2Ww0_2LwuQxpvo2HQ', # ProgrammingWithHarry
               'UCeVMnSShP_Iviwkknt83cww', # CodeWithHarry

               'UCZNNx4KYmCkwxCLdsHyWqQA', # Unacademy JEE
               'UC1ePkT4Kd_DrO1qK6-o0Qdg', # Unacademy Live - SSC Exams
               'UCABe2FgVNv2hgBeMu2mySVg', # Let's Crack UPSC CSE
               'UCdQwYksctqqiRwqp3PiJMWA', # Unacademy NEET
               'UCE6wlYZOldn_o25sXQzp0LA', # Unacademy - Shaurya
               'UCwFzHegPNQ8snevnmpzbv2w', # Unacademy CUET
               'UC4jZ2BALmfj-jaqVRTMqllg', # Unacademy Atoms
               'UCV1lbzgZNcYFAT3VHKOb7Cg', # Unacademy UPSC
               'UCa_AwYxequQ3Hek7qaITLgQ', # Unacademy Live - CSIR UGC NET
               'UCR3F3TPXHqXpotvmpyqXeQg', # Unacademy Class 11 & 12
               'UCFWCFYvqnAMT-jcCqTp_SlA', # Unacademy Computer Science
               'UC4SUQzurYVmGwgmfdn0yEVg', # Unacademy Class 9 & 10
               'UCPhY4iS6NNoQwlRLcUYu_tw', # Unacademy UPSC Hindi
               'UCjxQ7M1W2AL8hI_5JuxFYgA', # Unacademy - English Learning
               'UCrpELe0mluSZ8-2saJH5a0A', # Unacademy GATE - ME, PI, XE
               'UC2JNrw4j7Eo4R5cZXXn8rNw', # Unacademy CA Foundation Pro
               'UCBwwLyqbo6qTlDx5tS6Pl8w', # Unacademy CS Executive
               'UC7Y9ZsacCMna74LgYeVbDAQ', # Unacademy Class 6
               'UCONr9LnH0M3PA_VjchLNp1w', # Unacademy Live - IIT JAM
               'UCAgkv1L9uiXTskU8pl5TFDw', # Unacademy - लक्ष्य UPSC CSE
               
               'UCVJU_IChPMOe8RWkdVQjtfQ', # JEE Wallah
               'UCiGyWN6DEbnj2alu7iapuKQ', # Physics Wallah - Alakh Pandey
               'UCphU2bAGmw304CFAzy0Enuw', # Physics Wallah Foundation
               'UCD16eo98AXl-9T61Xd711kQ', # Competition Wallah
               'UC8zCnnfhz-dvIpVdZ1CheuA', # NCERT Wallah
               'UC8OCoGX9tmMneOzIFUTWq2Q', # Defence Wallah - NDA
               'UCko6jJyo-wUQmWhjGNAIrwg', # Physics Wallah - English
               'UCn5N03prvdzaZxEwq3AtHAw', # Commerce Wallah by PW

               'UCcabW7890RKJzL968QWEykA', # CS50
               'UCfzlCWGWYyIQ0aLC5w48gBQ', # sentdex

               'UCkw4JCwteGrDHIsyIIKo4tQ', # edureka!
               'UCywyZ4r4FTKExrPl9rf-ggw', # edureka! Hindi
               
               # Some Tech Channels
               'UCXUJJNoP1QupwsYIWFXmsZg', # Tech Burner
               'UCaSM4GqhbaVmRT7fmmFmR1w', # TechGumbo
               'UCXuqSBlHAE6Xw-yeJA0Tunw', # Linus Tech Tips
               'UCEPL07qzVsOcHd3sMUws65g', # Trakin Tech
               'UC1tVU8H153ZFO9eRsxdJlhA' # Technology Gyan
              ]

youtube = build('youtube', 'v3', developerKey=api_key)

Function to get channel status

In [66]:
def get_channel_stats(youtube, channel_ids):
    all_data = []
    request = youtube.channels().list(
                part='snippet,contentDetails,statistics',
                id=','.join(channel_ids))
    response = request.execute() 
    
    for i in range(len(response['items'])):
        data = dict(Channel_name = response['items'][i]['snippet']['title'],
                    Description = response['items'][i]['snippet']['description'],
                    playlist_id = response['items'][i]['contentDetails']['relatedPlaylists']['uploads']
                    )
        all_data.append(data)
    
    return all_data

In [67]:
channel_statistics = get_channel_stats(youtube, channel_ids)

In [68]:
channel_data = pd.DataFrame(channel_statistics)

In [72]:
# Change channel name here
channel_name = input("Enter Channel Name")
playlist_id = channel_data.loc[channel_data['Channel_name']==channel_name, 'playlist_id'].iloc[0]

In [73]:
def get_video_ids(youtube, playlist_id):
    
    request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50)
    response = request.execute()
    
    video_ids = []
    
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part='contentDetails',
                        playlistId = playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
            response = request.execute()
    
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
            
            next_page_token = response.get('nextPageToken')
        
    return video_ids

In [74]:
video_ids = get_video_ids(youtube, playlist_id)

In [75]:
video_ids

['38aggdt_ZAU',
 'oOmwUYujfwk',
 'cBvKoDNwvYM',
 'xWfzVOxNAOg',
 '3fOQjlpltm0',
 '00y5sEsQLKk',
 'YSH_j0cXFWA',
 'tpZE9iw0vZo',
 'u8Mh-GlbnoQ',
 'T9aEJFInu0s',
 'YQczx_qtHqE',
 'fQZZE2Y-fNk',
 'dSyzsmqfgyQ',
 'JqXeRe9U4g0',
 '6AV65wsYry4',
 'jEIIB4xLKYU',
 'EVvH9tugf4k',
 '0vDuwIBNPRc',
 'IV1G5G28Jbw',
 'RANXIdtE_Pg',
 'ekKS7dOYmvU',
 '1WNmrc0o5rg',
 'x1qLYnXI8v4',
 'NdbmQl0aVms',
 'oBpHnagZAzk',
 'wqQxNltTJ-8',
 'vEAvgw5QVcQ',
 'hhqFiPZC-hE',
 'khS1D_QQZrY',
 'WbpAVIIIK0M',
 'b5CqRyCPnq8',
 'qgFnT0Hrons',
 '4q0eAUVdcs8',
 'WGGM2jPzlIc',
 'MHCuVRtrGGE',
 'HMuyIsQxB8E',
 'g6ezBIhPJ-I',
 'FiQF1cBShDw',
 'ewMSVtxAvIM',
 'xzwfGHiBP3Q',
 'q2jfCnfkrSo',
 'Wda61b9tg7s',
 '4_BmohysAZ4',
 'gZyxNYB1qGM',
 'sLtjs3DDLWE',
 '2JWibjbiyIk',
 'W0M5Zj_T-mc',
 'RtlnhIxybGU',
 '-F_z3Zh3YJA',
 'F-sHjh9Jfbw',
 'UXiu1mtOqVU',
 'KZ6qj3zspBE',
 'prmlaCsO3l0',
 'dFe3x6MUOVQ',
 'qb9ctQ4lnrY',
 'cqwc6HQborQ',
 '4kth9Ol2nPw',
 'P7MAbI2BYWw',
 '7C-8TtolEzA',
 'WABGmd5vmFE',
 'pEVixQA_0e4',
 '_5mh54IW-UU',
 '0mr02n

In [76]:
def get_video_details(youtube, video_ids):
    all_video_stats = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
                    part='snippet,statistics',
                    id=','.join(video_ids[i:i+50]))
        response = request.execute()
        
        for video in response['items']:
            video_stats = dict(Title = video['snippet']['title'],
                               Description = video['snippet']['description']
                               )
            all_video_stats.append(video_stats)
            
    
    return all_video_stats

In [77]:
video_details = get_video_details(youtube, video_ids)

In [78]:
video_data = pd.DataFrame(video_details)

In [79]:
video_data

,Title,Description
0,🔴 Red Light Green Light 123! 🟢| KIIYII KARAOKE...,KiiYii and friends are running towards the fin...
1,Twinkle Twinkle Little Star KARAOKE! | CoComel...,Can JJ and friends help Twinkle? Cody and Nina...
2,🥦 Yes Yes Vegetables KARAOKE! 🥦| BEST Nursery ...,Enjoy this compilation of the very best kids s...
3,The Teacher Song KARAOKE! | CoComelon Nursery ...,We love teachers! Take time to say thank you ...
4,🧼 Bath Song KARAOKE! 🧼| 1 HOUR BEST OF Little ...,Enjoy this compilation of the best kids songs ...
...,...,...
751,Colour Fish Song| Nursery Rhyme For Babies | L...,Subscribe!\n\nhttps://www.youtube.com/channel/...
752,Colour Song | Nursery Rhyme For Babies | Lear...,Subscribe!\n\nhttps://www.youtube.com/channel/...
753,12 what shall we do | Nursery Rhyme For Babies...,Subscribe!\n\nhttps://www.youtube.com/channel/...
754,12345 Once I Caught A Fish Alive | Nursery Rhy...,Subscribe!\n\nhttps://www.youtube.com/channel/...


In [80]:
# To csv
# file_name_csv = 'Video_Details('+channel_name+').csv'
# video_data.to_csv(file_name_csv)
# To txt
file_name_txt = 'Video_Details('+channel_name+').txt'
video_data.to_csv(file_name_txt)

In [81]:
import re
f1 = open('Video_Details('+channel_name+').txt','r', encoding="utf8")
lines = f1.read()
s1 = re.sub('http://\S+|https://\S+', ' ', lines)
s2 = re.sub('http[s]?://\S+', ' ', s1)
s3 = re.sub(r"http\S+", " ", s2)
f1.close()


f2 = open('Video_Details('+channel_name+').txt','w', encoding="utf8")
f2.write(s3)
f2.close()